# Barcelona... lawless city?

In August 2019, the alarm raised. The word was Barcelona crime was spiraling out of control, at least if you were to believe some of the articles published by the press. Sentences like 'Barcelona lives the most violent summer of the decade', 'Street violence overruns Barcelona' or 'Barcelona, out of control' could be read in the national newspapers. The news reached the international press, and even the US Consulate General issued a Security Alert on the matter.

Of course, with this news other comments also came along, debating the accuracy of this stories and even claiming hidden agendas behind them (yes, some of us love conspiranoia, I'm afraid). As a person born and raised in Barcelona, I was curious: were these claims really true? Was the insecurity feeling that high, and  was it supported by data?

## A thermometer on the conversation: Twitter

In order to analyze the impact that these topic had had on the public debate, I turned to Twitter. I wanted to understand the trend in the conversations about the violence in Barcelona, so I scrapped the social network looking for tweets published between 2017 and 2019 that contained the word Barcelona plus at least another word related to the subject of violence or crime:

In [1]:
from twitterscraper import query_tweets
import datetime as dt
import pandas as pd
import time

day_delta = dt.timedelta(days=1)
start_date = dt.date(2018, 1, 5)
final_date = dt.date(2017, 12, 29) ##dummy date to avoid running whole code

while start_date > final_date:
    begin_date = start_date - 7*day_delta
    end_date = start_date
    query = '"Barcelona" AND "violencia" OR "crimen" OR "inseguridad" OR "delincuencia" OR "agresion" OR "delito" OR "crim" OR "inseguretat" OR "delinqüència" OR "agressió" OR "delicte"'
    tweets = query_tweets(query=query, begindate = begin_date, enddate = end_date)
    print(len(tweets), f' were retrieved for {begin_date} to {end_date}')
    df = pd.DataFrame(t.__dict__ for t in tweets)
    df.to_csv(f'Tweets_{begin_date}_{end_date}.csv')
    start_date -= 7*day_delta
    time.sleep(60)

INFO: {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'}
INFO: queries: ['"Barcelona" AND "violencia" OR "crimen" OR "inseguridad" OR "delincuencia" OR "agresion" OR "delito" OR "crim" OR "inseguretat" OR "delinqüència" OR "agressió" OR "delicte" since:2017-12-29 until:2017-12-30', '"Barcelona" AND "violencia" OR "crimen" OR "inseguridad" OR "delincuencia" OR "agresion" OR "delito" OR "crim" OR "inseguretat" OR "delinqüència" OR "agressió" OR "delicte" since:2017-12-30 until:2017-12-31', '"Barcelona" AND "violencia" OR "crimen" OR "inseguridad" OR "delincuencia" OR "agresion" OR "delito" OR "crim" OR "inseguretat" OR "delinqüència" OR "agressió" OR "delicte" since:2017-12-31 until:2018-01-01', '"Barcelona" AND "violencia" OR "crimen" OR "inseguridad" OR "delincuencia" OR "agresion" OR "delito" OR "crim" OR "inseguretat" OR "delinqüència" OR "agressió" OR "delicte" since:2018-01-01 until:2018-01-02', '"Barcelona" AND "violencia" OR "crimen" OR "inseguridad

240  were retrieved for 2017-12-29 to 2018-01-05


Once I had all the conversations downloaded, I uploaded the data, cleaned it and plotted it. 

<img style="float: centre;" src="Screenshots/Tweets per week_2.png" width="700"/>

Some plotting on the amount of related tweets per week lead me to see what I already suspected: the increase in the conversations about the violence and crime in Barcelona during the summer of 2019 was very evident, reaching more than 4.000 tweets at during week 34. When analyzing the conversation trend, this amount of tweets per week was only exceeded during the October 2019 riots following the incarceration of the pro-independent Catalan leaders. The other event that nearly met that number of tweets on the topic was the terror attack that took place in summer 2017 in Las Ramblas.

## Checking the real crime numbers: official data

In order to check if the social alert was indeed based on real data, I turned to two official sources: the Ministerio de Interior and the Mossos d'Esquadra. Both organizations make some of their data available online through different websites.

For data related to the Ministerio de Interior I used https://estadisticasdecriminalidad.ses.mir.es/. This website contains statistical information related to the activity of the different security agencies of Spain, including crimes commited and solved in the whole territory. For my analysis I used the 'Balances trimestrales de criminalidad', a summary of crimes reported in Spain by city and type of crime.

The files made available for download are not exactly perfect, so I had some trouble exporting and importing the CSVs. In the end, I decided to go for the Excel file, which seemed to be easier to work with (all things considered) but I had to create some ad-hoc processing functions in order to obtain a correct dataframe for the data analysis, as the original file looked like this:


<img style="float: centre;" src="Screenshots/MI_XLS.JPG" width="500"/>

Moreover, a closer look at the data made me realize that the information provided for the second, third and fourth trimester  of the year was actually aggregated data (meaning that the number of reported crimes was the total numbers reported from the beggining of the year). Therefore, in order to get the number of crimes per trimester, I had to perform some basic math operations on all the tables. 

By the time I finished, though, I had a clean dataframe that contained information on the number of reported crimes by type, city, year and trimester.

In [2]:
spain_crime = pd.read_csv('Analysis/crimes_spain.csv')
spain_crime.drop(['Unnamed: 0'], axis=1, inplace=True)
spain_crime.head()

,crime,quantity,year,trimester,city
0,1.-Homicidios dolosos y asesinatos consumados,0,2019,1,Córdoba
1,2.-Homicidios dolosos y asesinatos en grad...,0,2019,1,Córdoba
2,3.-Delitos graves y menos graves de lesion...,18,2019,1,Córdoba
3,4.-Secuestro,1,2019,1,Córdoba
4,5.-Delitos contra la libertad e indemnidad...,32,2019,1,Córdoba


The other main source I used was the Mossos d'Esquadra (the Catalan police agency) open data website: https://mossos.gencat.cat/ca/els_mossos_desquadra/indicadors_i_qualitat/dades_obertes/cataleg_dades_obertes/. Similarly to the Ministerio de Interior, the Mossos d'Esquadra make available some data related to their organization and activity in this link. More precisely, they provide information on the number of crimes reported and solved by type, month and ABP (police station units).

The JSON provided for download was quite clean, though there were some missing values as well as some crime categories that did not appear consistently through the years and months downloaded in the dataframe. For the purpose of this analysis, I assumed that the crimes that were NaN or not specified were equal to 0, as there was no simple way for me to infere any other value.

In [3]:
catalunya_crime = pd.read_csv('Analysis/crimes_barcelona.csv')
catalunya_crime.drop(['Unnamed: 0'], axis=1, inplace=True)
catalunya_crime.head()

,month,month_name,year,RP,ABP,type_of_crime,number_of_crimes
0,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,Homicidi consumat,1
1,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,Homicidi temptativa,1
2,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De la usurpació de l'estat civil,2
3,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,Falsedats documentals,2
4,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,Falsificació de moneda i efectes timbrats,2


## Analysis on the criminality data

### Crime rate: evolution in time

My first thought, when trying to understand the reason of the sudden increase in the social alarm about Barcelona criminality, was that there had been also a sudden increase in the amount of crimes perpetrated in the city. To test my hypothesis, I ran a first comparison on the number of crimes per year in the city:

<img style="float: centre;" src="Screenshots/Crime evolution all.png" width="1200"/>

What I could see at first glance was that, though there had indeed been an increment in the amount of crimes reported per year, the relative increase for 2019 (3.02%) was not nearly as high as the one for the previous year (17.5%) or the year before that (7.82%). 

That took me to my next hypothesis, which was that within the same year, there had been a significantly increase in the number of reported crimes during the summer months. While I checked this possibility I realise that, though the criminality indeed seemed to grow as the summer approached, this was a trend that was repeated every year, therefore not justifying either the raise in the alarm I was looking to explain:

<img style="float: centre;" src="Screenshots/Trend per month_2016_2019_2.png" width="1200"/>

### Crime rate: comparison to Spain

Since my first approach did not lead to any significant conclusions, I decided to compare the criminality numbers in Barcelona to those of other big cities in Spain. My main goal here was to check if there was some significant difference that could provide me further insight on the reason for the increase in the news, or if on the contrary, the numbers were similar in every city, and maybe supporting the theory that there was some other hidden motive behind the criminality conversation.

In order to compare the crime numbers on the different cities, I took into account the variability in the population, as a direct comparison in crime numbers would not be accurate: it would be expected that a city with more habitants had also a higher number of crimes reported. 

To do so, I scrapped Wikipedia (https://es.wikipedia.org/wiki/Anexo:Municipios_de_Espa%C3%B1a_por_poblaci%C3%B3n) to get the most populated cities in Spain:

In [4]:
cities = pd.read_html('https://es.wikipedia.org/wiki/Anexo:Municipios_de_Espa%C3%B1a_por_poblaci%C3%B3n',header=0)[1]
cities.head()

,#,Nombre,Población(2019),Provincia,Comunidad autónoma
0,1,Madrid,3 266 126,Madrid,Comunidad de Madrid
1,2,Barcelona,1 636 762,Barcelona,Cataluña
2,3,Valencia,794 288,Valencia,Comunidad Valenciana
3,4,Sevilla,688 592,Sevilla,Andalucía
4,5,Zaragoza,674 997,Zaragoza,Aragón


With this information, I proceeded to calculate the ratio of crimes reported per 1000 habitants and year, according to the numbers provided by the Ministerio de Interior, and I was quite torned by the resuts... 

<img style="float: centre;" src="Screenshots/Crime per city.jpg" width="1200"/>

On one hand, I could not appreciate a pattern in the crime evolution that explained the media outburst on the crime topic (once again, the 2018 numbers seem more significant than the ones registered in 2019). On the other hand, though, the crime ratio in Barcelona was clearly higher than in any of the other 15 most populated cities in Spain, therefore leading to the conclusion that it was, in fact, a relatively dangerous city (when compared to the rest of the country).

### Crime rate: by neighbourhood and type of crime

As I had still not identified a trend that explained the increase in the conversation trend, though, I went back to the Mossos d'Esquadra data with a new question: was there maybe some qualitative data that explained the shift on the population's perception? Maybe there was more crime on different neighbourhoods? Or maybe the type of crime was different?

To answer the first question, I performed a comparison on the number of crimes reported per neighbourhood in the last three years:


<img style="float: centre;" src="Screenshots/Crime per neighbourhood.jpg" width="1200"/>

Plotting that information showed mainly two things. First of all, there was a very uneven distribution on the number of crimes reported depending on the neighbourhood, and Ciutat Vella and Eixample, two of the most touristy neighbourhoods, were clearly leading the race.

Secondly, there was indeed a steady increase in the number of crimes in most of the neighbourhoods. Though the crimes reported in Ciutat Vella during 2019 had decreased, in most of the other (residential) neighbourhoods the number was getting higher. Not significantly, but steadily higher.

As per the second indicator, I decided to check the relative increase per month on some crime categories that could be classified as "violent crimes". After all, most of the news sources were talking about an increase in the violence in Barcelona, so it could be the case that, even though the total number of crimes had not been significantly higher, the number of violent crimes had increased at a higher ratio.

With this idea in mind, I created a subset of the crimes that could be considered as violent, such as attempted murder or  assault and robbery, and plotted the number of crimes reported during the first half of 2018 and again the first half of 2019, trying to find a trend. The results were, once again, inconclusive but overwhelming:

<img style="float: centre;" src="Screenshots/Trend per crime type I.png" width="600"/>
<img style="float: centre;" src="Screenshots/Trend per crime type II.png" width="600"/>

As it can be appreciated in the image, there are no significant growths in the number of crimes from may-jun 2019 to july-august 2019. However, if we analyze the global numbers, we can definitely appreciate a great increase in the total number of crimes per month in most of the categories when compared to the historical numbers at the beggining of 2018.

### Conclusions

After analyzing the insights provided by the data, I reached two conclusions.

The first one was that the available numbers did not indeed provide a direct correlation that explained the sudden and intense alarm about the criminality in Barcelona. When standardizing the increase in the number of conversations related to Barcelona and crime on Twitter during the 2019 summer versus the increase in the number of crimes during the same period, we can appreciate that the increase in the conversations was 3.7 times higher.

In [5]:
from sklearn import preprocessing
import numpy as np

#### Crimes
## Get the total number of crimes from january to august 2019 and the total number of crimes for august 2019
total_crimes_jan_aug_2019 = pd.read_csv('Analysis/bcn_crime_clean.csv')
total_crimes_jan_aug_2019 = total_crimes_jan_aug_2019[total_crimes_jan_aug_2019.year == 2019]
total_crimes_jan_aug_2019 = total_crimes_jan_aug_2019[(total_crimes_jan_aug_2019.month == 1) | (total_crimes_jan_aug_2019.month == 2) | (total_crimes_jan_aug_2019.month == 3) | (total_crimes_jan_aug_2019.month == 4) | (total_crimes_jan_aug_2019.month == 5) | (total_crimes_jan_aug_2019.month == 6) | (total_crimes_jan_aug_2019.month == 7) | (total_crimes_jan_aug_2019.month == 8)]
total_crimes_aug_2019 = total_crimes_jan_aug_2019[(total_crimes_jan_aug_2019.month == 8)]

## Calculate the mean of crimes per month during january to august 2019  
total_crimes_jan_aug_2019 = total_crimes_jan_aug_2019.groupby('month', as_index = False).sum()
total_crimes_jan_aug_2019.drop(['year', 'day'], axis=1, inplace=True)
total_crimes_jan_aug_2019

## Create Scaler object
scaler = preprocessing.StandardScaler()

## Fit data on the scaler object
array_crimes = np.array(total_crimes_jan_aug_2019.number_of_crimes).reshape(-1, 1)
scaled_df = scaler.fit_transform(array_crimes)
total_crimes_jan_aug_2019['scaled_crimes'] = scaled_df

#### Tweets
### Get the total number of tweets from january to august 2019 and the total number of tweets for august 2019
total_tweets_jan_aug_2019 = pd.read_csv('Analysis/tweets_clean.csv')
total_tweets_jan_aug_2019 = total_tweets_jan_aug_2019[total_tweets_jan_aug_2019.year == 2019]
total_tweets_jan_aug_2019 = total_tweets_jan_aug_2019[(total_tweets_jan_aug_2019.month == 1) | (total_tweets_jan_aug_2019.month == 2) | (total_tweets_jan_aug_2019.month == 3) | (total_tweets_jan_aug_2019.month == 4) | (total_tweets_jan_aug_2019.month == 5) | (total_tweets_jan_aug_2019.month == 6) | (total_tweets_jan_aug_2019.month == 7) | (total_tweets_jan_aug_2019.month == 8)]
total_tweets_aug_2019 = total_tweets_jan_aug_2019[(total_tweets_jan_aug_2019.month == 8)]

## Calculate the mean of crimes per month during january to august 2019  
total_tweets_jan_aug_2019 = total_tweets_jan_aug_2019.groupby('month', as_index = False).count()
total_tweets_jan_aug_2019.drop(['year', 'day', 'timestamp'], axis=1, inplace=True)

# Create the Scaler object
scaler = preprocessing.StandardScaler()

# Fit your data on the scaler object
array_tweets = np.array(total_tweets_jan_aug_2019.tweet_id).reshape(-1, 1)
scaled_df = scaler.fit_transform(array_tweets)
total_tweets_jan_aug_2019['scaled_tweets'] = scaled_df

print('The deviation from the mean in number of tweets for august 2019 was', total_tweets_jan_aug_2019.loc[7, 'scaled_tweets'])
print('The deviation from the mean in number of crimes for august 2019 was', total_crimes_jan_aug_2019.loc[7, 'scaled_crimes'])
print('The ratio between the two metrics is', total_tweets_jan_aug_2019.loc[7, 'scaled_tweets']/total_crimes_jan_aug_2019.loc[7, 'scaled_crimes'])

The deviation from the mean in number of tweets for august 2019 was 2.35581381975219
The deviation from the mean in number of crimes for august 2019 was 0.6349231127288472
The ratio between the two metrics is 3.710392286125948


The second one would be that, even though the alarm could be considered exaggerated as we focus on a single moment in time, if we consider the criminality trend in the city it can be denied that there has been a significant increase in the crime rate according to most of the analyzed features. 

### Notes

More info on data aquisition here: 

See presentation here: https://docs.google.com/presentation/d/1LTJ_KnBUSqusGuRzXS0tk5WCZ0Ko6R8C-oeUi3KRGKM/edit?usp=sharing